<a href="https://colab.research.google.com/github/shujuan12/Data_management_group10/blob/main/group_project_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home assignment

* Author: Romain Tavenard (@rtavenar)
* License: CC-BY-NC-SA

A home assignment from a course on Deep Learning at EDHEC.

## Problem statement

The dataset we are interested in here is called "CIFAR10". It is described [in this page](https://keras.io/api/datasets/cifar10/).

You should load the data, **select only 5,000 samples out of the total 50,000 ones**, and preprocess it if needed.
You should compare several candidate neural network architectures, and make a decision about which is best for the task at hand.
You should be explicit about the indicator(s) you base your decision on.

Finally, as a bonus, you could try to evaluate whether it is better to:
* train a model from scratch on this dataset alone ;
* use a large model that was pre-trained on ImageNet ;
* pre-train a model on another dataset called [CIFAR100](https://keras.io/api/datasets/cifar100/) and fine-tune it on CIFAR10.

## Deadline

Deadline for this home assignment is February 28th, 2025.
You should use the link on moodle to hand in your assignment.
A single ipynb file should be provided,
with execution traces.
This assignment is to be done **by groups of three, at most** and names of all students should be included in the file name.

## Data loading

You can use the dedicated `keras` utility to load this dataset: <https://keras.io/api/datasets/cifar10/>

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# 1. 固定随机种子，保证结果一致
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# 2. 加载 CIFAR-10 数据
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 3. 从 50,000 张图片中**随机选择 5,000 张**
sample_indices = np.random.choice(x_train.shape[0], 5000, replace=False)  # 不放回抽样
x_sampled = x_train[sample_indices]
y_sampled = y_train[sample_indices]

# 3. 归一化（像素值 0-255 -> 0-1）
x_sampled = x_sampled.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 4. 划分训练集和测试集（80% 训练，20% 测试）
x_train_sub, x_val_sub, y_train_sub, y_val_sub = train_test_split(
    x_sampled, y_sampled, test_size=0.2, random_state=seed
)

# 5. 将标签转换为 one-hot 编码（适用于 Softmax 分类）
y_train_sub = keras.utils.to_categorical(y_train_sub, 10)
y_val_sub = keras.utils.to_categorical(y_val_sub, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 输出数据形状，确保正确
print("训练集:", x_train_sub.shape, y_train_sub.shape)
print("验证集:", x_val_sub.shape, y_val_sub.shape)
print("测试集:", x_test.shape, y_test.shape)


训练集: (4000, 32, 32, 3) (4000, 10)
验证集: (1000, 32, 32, 3) (1000, 10)
测试集: (10000, 32, 32, 3) (10000, 10)


##1.Training Strategy-train a model from scratch on this dataset alone

设计思路：

2 层卷积 + 池化，适合小数据集，计算量低，防止过拟合。
ReLU 作为默认激活函数，提高非线性表达能力。

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 构建 Simple CNN 模型
def simple_cnn(input_shape=(32, 32, 3), num_classes=10):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# 初始化模型
model = simple_cnn()

# 编译模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练模型
history = model.fit(x_train_sub, y_train_sub,
                    epochs=10,
                    batch_size=64,
                    validation_data=(x_val_sub, y_val_sub))

In [ ]:
# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"最终测试集准确率: {test_acc:.4f}")

设计思路：

4 层卷积 + 池化，增加特征提取能力。
批归一化（Batch Normalization），加快训练并稳定收敛。
Dropout（随机丢弃），防止过拟合。

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 构建 Deep CNN 模型
def deep_cnn(input_shape=(32, 32, 3), num_classes=10):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),  # 防止过拟合
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# 初始化模型
model = deep_cnn()

# 编译模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练模型
history = model.fit(x_train_sub, y_train_sub,
                    epochs=20,
                    batch_size=64,
                    validation_data=(x_val_sub, y_val_sub))

# 评估模型
test_loss, test_acc = model.evaluate(x_test_sub, y_test_sub)
print(f"最终测试集准确率: {test_acc:.4f}")


Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 53s 358ms/step - accuracy: 0.2395 - loss: 2.5642 - val_accuracy: 0.1120 - val_loss: 2.5036
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 356ms/step - accuracy: 0.3972 - loss: 1.7042 - val_accuracy: 0.1480 - val_loss: 3.4426
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 354ms/step - accuracy: 0.4732 - loss: 1.4641 - val_accuracy: 0.0890 - val_loss: 6.2937
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 40s 336ms/step - accuracy: 0.5540 - loss: 1.2266 - val_accuracy: 0.0890 - val_loss: 6.7948
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 357ms/step - accuracy: 0.6707 - loss: 0.9623 - val_accuracy: 0.0890 - val_loss: 7.6902
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 340ms/step - accuracy: 0.7220 - loss: 0.7959 - val_accuracy: 0.1210 - val_loss: 4.6952
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 349ms/step - accuracy: 0.7934 - loss: 0.6045 - val_accuracy: 0.3060 - val_loss: 2.7190
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 353ms/step - accuracy: 0.8319 - loss: 0.4968 - val_accu

设计思路：

引入残差连接（skip connection），防止梯度消失。
适合较深的网络，提高训练稳定性。

In [ ]:
def residual_block(x, filters):
    shortcut = x  # 直接连接

    x = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)

    x = layers.Add()([x, shortcut])  # 残差连接
    x = layers.Activation('relu')(x)
    return x

def resnet_cnn(input_shape=(32, 32, 3), num_classes=10):
    inputs = keras.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = residual_block(x, 32)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = residual_block(x, 64)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()
    x = layers.Dense(256, activation='relu')
    x = layers.Dropout(0.5)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model


In [8]:
def inception_module(x, filters):
    conv1x1 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(x)

    conv3x3 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    conv3x3 = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(conv3x3)

    conv5x5 = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    conv5x5 = layers.Conv2D(filters, (5, 5), padding='same', activation='relu')(conv5x5)

    pooled = layers.MaxPooling2D((3, 3), strides=1, padding='same')(x)
    pooled = layers.Conv2D(filters, (1, 1), padding='same', activation='relu')(pooled)

    output = layers.concatenate([conv1x1, conv3x3, conv5x5, pooled], axis=-1)
    return output

def inception_cnn(input_shape=(32, 32, 3), num_classes=10):
    inputs = keras.Input(shape=input_shape)

    x = inception_module(inputs, 32)
    x = layers.MaxPooling2D((2, 2))(x)

    x = inception_module(x, 64)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()
    x = layers.Dense(256, activation='relu')
    x = layers.Dropout(0.5)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model


##2.Training Strategy-pre-train a model on another dataset called CIFAR100 and fine-tune it on CIFAR10.